<a href="https://colab.research.google.com/github/Emmanuel-kabu/Timeseries/blob/main/Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=43ac680bbf3e8dd7dccf8e71276eae7695c171bea8f60ea5b1110b26a92b3b25
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4185k      0  0:00:19  0:00:19 --:--:-- 3655k


In [4]:
!dir


aclImdb  aclImdb_v1.tar.gz  sample_data


In [5]:
import os
path = 'aclImdb'
print(os.listdir(path))

['README', 'imdb.vocab', 'test', 'imdbEr.txt', 'train']


In [6]:
train_path = os.path.join(path, 'train')
print(os.listdir(train_path))


['neg', 'urls_unsup.txt', 'urls_pos.txt', 'pos', 'unsupBow.feat', 'urls_neg.txt', 'unsup', 'labeledBow.feat']


In [7]:
test_path = os.path.join(path, 'test')
print(os.listdir(test_path))
for i in os.listdir(test_path):
    print(i)

['neg', 'urls_pos.txt', 'pos', 'urls_neg.txt', 'labeledBow.feat']
neg
urls_pos.txt
pos
urls_neg.txt
labeledBow.feat


In [10]:
 !rm -rf aclImdb/train/unsup

In [11]:
!cat aclImdb/train/pos/4077_10.txt


I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [12]:
import pathlib, shutil, random
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'
for category in ('neg', 'pos'):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)


In [13]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size = batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size = batch_size )
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size = batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [14]:
for targets, inputs in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)

inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targets.shape: (32,)
targets.dtype: <dtype: 'string'>
inputs.shape: (32,)
inputs.dtype: <dtype: 'int32'>
targ

Preproceesing the the train_ds


In [16]:
from tensorflow.keras.layers import TextVectorization

In [18]:
text_vectorization = TextVectorization(max_tokens = 20000, output_mode='multi_hot')
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)


Building the model

In [20]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers
inputs = keras.Input(shape=(20000,),)
x = layers.Dense(16, activation='relu')(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [26]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])


In [22]:
callbacks = [keras.callbacks.ModelCheckpoint("binary_1gram.keras",save_best_only=True)]

In [27]:
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 67s 21ms/step - accuracy: 0.7774 - loss: 0.4960 - val_accuracy: 0.8686 - val_loss: 0.3150
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.8999 - loss: 0.2761 - val_accuracy: 0.8768 - val_loss: 0.2991
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9164 - loss: 0.2396 - val_accuracy: 0.8846 - val_loss: 0.3053
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9279 - loss: 0.2139 - val_accuracy: 0.8838 - val_loss: 0.3175
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9309 - loss: 0.2130 - val_accuracy: 0.8848 - val_loss: 0.3185
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9339 - loss: 0.2120 - val_accuracy: 0.8802 - val_loss: 0.3345
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9372 - loss: 0.1980 - val_accuracy: 0.8840 - val_loss: 0.3394
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9387 - loss: 0.1928 - val_accu

In [31]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.8779 - loss: 0.3011
Test acc: 0.877
